# Let's Begin

## Imports

In [20]:
import tensorflow as tf
from tensorflow.keras import mixed_precision
from tensorflow.keras import layers, Model
import numpy as np
import matplotlib.pyplot as plt
import os

In [21]:
# print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
# physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [22]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # Disable GPU

In [23]:
tf.keras.backend.clear_session() 

In [24]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

In [25]:
np.random.seed(42)
tf.random.set_seed(42)

In [26]:
LOW_RES_PATH = "./LR/"
HIGH_RES_PATH = "./HR/"

In [27]:
EPOCHS = 10
BATCH_SIZE = 1
HR_SHAPE = (2040, 2040, 3)
LR_SHAPE = (192, 256, 3)
LEARNING_RATE = 1e-4

In [28]:
def preprocess_image(image_path, target_size):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, target_size)
    image = tf.cast(image, tf.float16) / 127.5 - 1.0  # Normalize to [-1, 1]
    return image

In [29]:
def load_dataset(lr_path, hr_path, lr_shape, hr_shape, batch_size):
    lr_files = tf.data.Dataset.list_files(os.path.join(lr_path, "*.png"), shuffle=True)
    hr_files = tf.data.Dataset.list_files(os.path.join(hr_path, "*.png"), shuffle=True)

    lr_images = lr_files.map(
        lambda x: preprocess_image(x, lr_shape[:2]), num_parallel_calls=tf.data.AUTOTUNE
    )
    hr_images = hr_files.map(
        lambda x: preprocess_image(x, hr_shape[:2]), num_parallel_calls=tf.data.AUTOTUNE
    )

    dataset = tf.data.Dataset.zip((lr_images, hr_images))
    dataset = dataset.shuffle(buffer_size=256)  # Reduce shuffle buffer
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)  # Prefetch for performance
    return dataset

In [30]:
dataset = load_dataset(LOW_RES_PATH, HIGH_RES_PATH, LR_SHAPE, HR_SHAPE, BATCH_SIZE)

In [31]:
def build_generator(input_shape=(192, 256, 3)):
    inputs = layers.Input(shape=input_shape)

    # Initial Convolution Block
    x = layers.Conv2D(64, (9, 9), padding="same")(inputs)
    x = layers.PReLU()(x)
    x = layers.Conv2D(64, (3, 3), padding="same")(x)
    residual = x

    # Residual Blocks
    for _ in range(16):
        x = layers.Conv2D(64, (3, 3), padding="same")(residual)
        x = layers.BatchNormalization()(x)
        x = layers.PReLU()(x)
        x = layers.Conv2D(64, (3, 3), padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.add([x, residual])

    # Upsampling Layers
    for _ in range(2):  # 2x2 Upscale (4x in total)
        x = layers.Conv2D(256, (3, 3), padding="same")(x)
        x = layers.UpSampling2D(size=(2, 2))(x)
        x = layers.PReLU()(x)

    # Final Output Block (Change filters to 3 for RGB output)
    outputs = layers.Conv2D(3, (9, 9), activation="tanh", padding="same")(x)
    outputs = layers.Lambda(lambda outputs: tf.image.resize(outputs, HR_SHAPE[:2]))(outputs)

    return Model(inputs, outputs)

generator = build_generator(LR_SHAPE)
generator.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 192, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cast (Cast)         │ (None, 192, 256,  │          0 │ input_layer[0][0] │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 192, 256,  │     15,616 │ cast[0][0]        │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ p_re_lu (PReLU)     │ (None, 192, 256,  │  3,145,728 │ conv2d[0][0]      │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 192, 256,  │     36,928 │ p_re_lu[0][0]     │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_32 (Conv2D)  │ (None, 192, 256,  │     36,928 │ conv2d_1[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 192, 256,  │        256 │ conv2d_32[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ p_re_lu_16 (PReLU)  │ (None, 192, 256,  │  3,145,728 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_33 (Conv2D)  │ (None, 192, 256,  │     36,928 │ p_re_lu_16[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 192, 256,  │        256 │ conv2d_33[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_15 (Add)        │ (None, 192, 256,  │          0 │ batch_normalizat… │
│                     │ 64)               │            │ conv2d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_34 (Conv2D)  │ (None, 192, 256,  │    147,712 │ add_15[0][0]      │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d       │ (None, 384, 512,  │          0 │ conv2d_34[0][0]   │
│ (UpSampling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ p_re_lu_17 (PReLU)  │ (None, 384, 512,  │ 50,331,648 │ up_sampling2d[0]… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_35 (Conv2D)  │ (None, 384, 512,  │    590,080 │ p_re_lu_17[0][0]  │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_1     │ (None, 768, 1024, │          0 │ conv2d_35[0][0]   │
│ (UpSampling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ p_re_lu_18 (PReLU)  │ (None, 768, 1024, │ 201,326,5… │ up_sampling2d_1[

 Total params: 258,876,611 (987.54 MB)

 Trainable params: 258,876,355 (987.53 MB)

 Non-trainable params: 256 (1.00 KB)

In [32]:
def build_discriminator(input_shape=(128, 128, 3)):
    inputs = layers.Input(shape=input_shape)

    # Convolutional Blocks
    x = layers.Conv2D(64, (3, 3), strides=1, padding="same")(inputs)
    x = layers.LeakyReLU(alpha=0.2)(x)

    for filters in [64, 128, 128, 256, 256, 512, 512]:
        x = layers.Conv2D(filters, (3, 3), strides=2, padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.LeakyReLU(alpha=0.2)(x)

    # Global Pooling Layer instead of Flatten
    x = layers.GlobalAveragePooling2D()(x)

    # Dense Layers
    x = layers.Dense(1024)(x)  # Reduced size
    x = layers.LeakyReLU(alpha=0.2)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)

    return Model(inputs, outputs)

discriminator = build_discriminator(HR_SHAPE)
discriminator.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 2040, 2040, 3)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cast_1 (Cast)                   │ (None, 2040, 2040, 3)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_37 (Conv2D)              │ (None, 2040, 2040, 64) │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 2040, 2040, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_38 (Conv2D)              │ (None, 1020, 1020, 64) │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_32          │ (None, 1020, 1020, 64) │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 1020, 1020, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_39 (Conv2D)              │ (None, 510, 510, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_33          │ (None, 510, 510, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 510, 510, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_40 (Conv2D)              │ (None, 255, 255, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_34          │ (None, 255, 255, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 255, 255, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_41 (Conv2D)              │ (None, 128, 128, 256)  │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_35          │ (None, 128, 128, 256)  │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 128, 128, 256)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_42 (Conv2D)              │ (None, 64, 64, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_36          │ (None, 64, 64, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_5 (LeakyReLU)       │ (None, 64, 64, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_43 (Conv2D)              │ (None, 32, 32, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_37          │ (None, 32, 32, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_6 (LeakyReLU)       │ (None, 32, 32, 512)    │             

 Total params: 5,219,137 (19.91 MB)

 Trainable params: 5,215,425 (19.90 MB)

 Non-trainable params: 3,712 (14.50 KB)

In [33]:
# Pre-trained VGG model for perceptual loss
vgg = tf.keras.applications.VGG19(include_top=False, weights="imagenet", input_shape=HR_SHAPE)
vgg.trainable = False
vgg = Model(vgg.input, vgg.layers[10].output)  # Use intermediate layer for feature extraction

In [34]:
binary_cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=False)

In [35]:
def perceptual_loss(hr, sr):
    sr_features = vgg(sr)
    hr_features = vgg(hr)
    return tf.reduce_mean(tf.square(hr_features - sr_features))

In [36]:
# Optimizers
g_optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
d_optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

In [37]:
@tf.function
def train_step(lr_batch, hr_batch):
    with tf.GradientTape(persistent=True) as tape:
        # Forward pass
        fake_hr = generator(lr_batch, training=True)
        
        # Cast both the fake_hr and hr_batch to float32 for loss calculation
        fake_hr = tf.cast(fake_hr, dtype=tf.float32)
        hr_batch = tf.cast(hr_batch, dtype=tf.float32)
        
        # Calculate loss (Example: using L1 loss)
        g_loss = tf.reduce_mean(tf.abs(fake_hr - hr_batch))  # Example loss
        
        # Example of discriminator loss
        d_loss_real = discriminator(hr_batch, training=True)
        d_loss_fake = discriminator(fake_hr, training=True)
        d_loss = 0.5 * (tf.reduce_mean(d_loss_real) + tf.reduce_mean(d_loss_fake))
        
    # Calculate gradients
    gradients_of_generator = tape.gradient(g_loss, generator.trainable_variables)
    gradients_of_discriminator = tape.gradient(d_loss, discriminator.trainable_variables)
    
    # Apply gradients
    g_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    d_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    # Delete the tape to free resources after use
    del tape

    return d_loss, g_loss

In [38]:
# Training loop
for epoch in range(EPOCHS):
    i=0
    for lr_batch, hr_batch in dataset:
        i+=1
        d_loss, g_loss = train_step(lr_batch, hr_batch)
        print(f"Image: {i}, d_loss: {d_loss:.4f}, g_loss: {g_loss:.4f}", end='\r')

    if epoch % 1 == 0:
        print(f"Epoch {epoch}/{EPOCHS} | D Loss: {d_loss} | G Loss: {g_loss:.4f}")

    # Save model periodically
    if epoch % 1 == 0:
        generator.save(f"generator_epoch_{epoch}.h5")

2025-01-08 22:29:15.530085: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 32614907904 exceeds 10% of free system memory.
2025-01-08 22:29:15.533871: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 32614907904 exceeds 10% of free system memory.
2025-01-08 22:29:15.535855: W tensorflow/core/framework/op_kernel.cc:1841] OP_REQUIRES failed at conv_grad_filter_ops.cc:380 : RESOURCE_EXHAUSTED: OOM when allocating tensor with shape[1,786432,20736] and type half on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
2025-01-08 22:29:15.536995: W tensorflow/core/framework/op_kernel.cc:1841] OP_REQUIRES failed at conv_grad_input_ops.h:598 : RESOURCE_EXHAUSTED: OOM when allocating tensor with shape[1,786432,20736] and type half on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
2025-01-08 22:29:15.702443: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: RESOURCE_EX

ResourceExhaustedError: Graph execution error:

Detected at node gradient_tape/functional_1/conv2d_36_1/convolution/Conv2DBackpropFilter defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/ragnar/.local/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/ragnar/.local/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/ragnar/.local/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/ragnar/.local/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.12/asyncio/base_events.py", line 641, in run_forever

  File "/usr/lib/python3.12/asyncio/base_events.py", line 1987, in _run_once

  File "/usr/lib/python3.12/asyncio/events.py", line 88, in _run

  File "/home/ragnar/.local/lib/python3.12/site-packages/ipyker

In [ ]:
generator.save("srgan_generator.h5")

In [79]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image



# Load a low-resolution sample image for prediction
def load_image(path, target_size):
    img = preprocess_image(path, target_size)
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Generate predictions
def generate_high_res(model, lr_image):
    prediction = model.predict(lr_image)
    prediction+=1.0
    prediction/=2.0
    prediction = prediction.reshape(HR_SHAPE)
    return prediction

# Visualize the output
def plot_images(lr_image, hr_image):
    print(lr_image.shape)
    # lr_image = lr_image[0]  # Remove batch dimension
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    axes[0].imshow(lr_image)
    axes[0].set_title("Low-Resolution Input")
    axes[0].axis("off")
    
    axes[1].imshow(hr_image)
    axes[1].set_title("High-Resolution Output")
    axes[1].axis("off")
    
    plt.tight_layout()
    plt.show()




In [ ]:
# Path to your low-resolution image
lr_image_path = "LR/0059.png"  # Replace with the actual path
lr_image = load_image(lr_image_path, LR_SHAPE[:2])  # Adjust size as per your model's input

# Generate high-resolution image
hr_image = generate_high_res(generator, lr_image)


lr_image = lr_image.reshape(LR_SHAPE).astype(np.float32)
lr_image+=1.0
lr_image/=2.0
# Plot the images
plot_images(lr_image, hr_image)


In [ ]:
# Path to your low-resolution image
lr_image_path = "LR/0009.png"  # Replace with the actual path
lr_image = load_image(lr_image_path, LR_SHAPE[:2])  # Adjust size as per your model's input

# Generate high-resolution image
hr_image = generate_high_res(generator, lr_image)


lr_image = lr_image.reshape(LR_SHAPE).astype(np.float32)
lr_image+=1.0
lr_image/=2.0
# Plot the images
plot_images(lr_image, hr_image)


In [ ]:
# Path to your low-resolution image
lr_image_path = "LR/0099.png"  # Replace with the actual path
lr_image = load_image(lr_image_path, LR_SHAPE[:2])  # Adjust size as per your model's input

# Generate high-resolution image
hr_image = generate_high_res(generator, lr_image)


lr_image = lr_image.reshape(LR_SHAPE).astype(np.float32)
lr_image+=1.0
lr_image/=2.0
# Plot the images
plot_images(lr_image, hr_image)


In [ ]:
# Path to your low-resolution image
lr_image_path = "LR/0159.png"  # Replace with the actual path
lr_image = load_image(lr_image_path, LR_SHAPE[:2])  # Adjust size as per your model's input

# Generate high-resolution image
hr_image = generate_high_res(generator, lr_image)


lr_image = lr_image.reshape(LR_SHAPE).astype(np.float32)
lr_image+=1.0
lr_image/=2.0
# Plot the images
plot_images(lr_image, hr_image)
